# 코루틴

- 코루틴(coroutine)은 cooperative routine를 의미하는데 서로 협력하는 루틴이라는 뜻입니다. 
- 즉, 메인 루틴과 서브 루틴처럼 종속된 관계가 아니라 서로 대등한 관계이며 특정 시점에 상대방의 코드를 실행합니다.
- 코루틴은 함수가 종료되지 않은 상태에서 메인 루틴의 코드를 실행한 뒤 다시 돌아와서 코루틴의 코드를 실행합니다. 따라서 코루틴이 종료되지 않았으므로 코루틴의 내용도 계속 유지됩니다.
- 함수의 코드를 실행하는 지점을 진입점(entry point)이라고 하는데, 코루틴은 진입점이 여러 개인 함수입니다.

- 코루틴은 함수가 종료되지 않은 상태에서 값을 주고 받을 수 있는 함수이며 이 과정에서 현재 코드의 실행을 대기하고 상대방의 코드를 실행한다는 점이 중요합니다.
- 보통 코루틴은 시간이 오래 걸리는 작업을 분할하여 처리할 때 사용하는데 주로 파일 처리, 네트워크 처리 등에 활용합니다.

## 1. 코루틴에 값 보내기

- 코루틴은 제너레이터의 특별한 형태입니다.
- 제너레이터는 yield로 값을 발생시켰지만 코루틴은 yield로 값을 받아올 수 있습니다.

```
코루틴객체.send(값)
변수 = (yield)
```

In [2]:
def num_coroutine():
    while True:        # 코루틴을 계속 유지하기 위해 무한 루프 사용
        x = (yield)    # 코루틴 바깥에서 값을 받아옴, yield를 괄호로 묶어야 함
        print(x)
 
co = num_coroutine()
next(co)      # 코루틴 안의 yield까지 코드 실행(최초 실행)
 
co.send(1)    # 코루틴에 숫자 1을 보냄
co.send(2)    # 코루틴에 숫자 2을 보냄
co.send(3)    # 코루틴에 숫자 3을 보냄

1
2
3


- send로 코루틴의 코드를 최초로 실행하기
  - 코루틴객체.send(None)과 같이 send 메서드에 None을 지정해도 코루틴의 코드를 최초로 실행할 수 있습니다.

## 2. 코루틴 바깥으로 값 전달하기

- (yield 변수) 형식으로 yield에 변수를 지정한 뒤 괄호로 묶어주면 값을 받아오면서 바깥으로 값을 전달합니다.
- 그리고 yield를 사용하여 바깥으로 전달한 값은 next 함수(\_\_next__ 메서드)와 send 메서드의 반환값으로 나옵니다.

```
변수 = (yield 변수)
변수 = next(코루틴객체)
변수 = 코루틴객체.send(값)
```

In [8]:
def sum_coroutine():
    total = 0
    while True:        
        x = (yield total)  # 코루틴 바깥에서 값을 받아오면서 바깥으로 값을 전달
        total += x
 
co = sum_coroutine()
print(next(co))      # 0: 코루틴 안의 yield까지 코드를 실행하고 코루틴에서 나온 값 출력
 
print(co.send(1))    # 1: 코루틴에 숫자 1을 보내고 코루틴에서 나온 값 출력
print(co.send(2))    # 3: 코루틴에 숫자 2를 보내고 코루틴에서 나온 값 출력
print(co.send(3))    # 6: 코루틴에 숫자 3을 보내고 코루틴에서 나온 값 출력

0
1
3
6


### ※ 제너레이터와 코루틴의 차이점
  - 제너레이터는 next 함수(\_\_next__ 메서드)를 반복 호출하여 값을 얻어내는 방식
  - 코루틴은 next 함수(\_\_next__ 메서드)를 한 번만 호출한 뒤 send로 값을 주고 받는 방식

## 3. 코루틴을 종료하고 예외 처리하기

- 보통 코루틴은 실행 상태를 유지하기 위해 while True:를 사용해서 끝나지 않는 무한 루프로 동작합니다. 
- 만약 코루틴을 강제로 종료하고 싶다면 close 메서드를 사용합니다.

```
코루틴객체.close()
```

- 파이썬 스크립트가 끝나면 코루틴도 끝나기 때문에 close를 사용하지 않은 것과 별 차이가 없습니다. 
- 하지만 close는 코루틴의 종료 시점을 알아야 할 때 사용하면 편리합니다.

In [4]:
def num_coroutine():
    while True:
        x = (yield)
        print(x, end=' ')
        
co = num_coroutine()
next(co)

for i in range(20):
    co.send(i)
    
co.close()  # 코루틴 종료

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

- GeneratorExit 예외 처리하기
  - 코루틴 객체에서 close 메서드를 호출하면 코루틴이 종료될 때 GeneratorExit 예외가 발생합니다. 
  - 따라서 이 예외를 처리하면 코루틴의 종료 시점을 알 수 있습니다.

In [5]:
def num_coroutine():
    try:
        while True:
            x = (yield)
            print(x, end=' ')
    except GeneratorExit:  # 코루틴이 종료 될 때 GeneratorExit 예외 발생
        print()
        print('코루틴 종료')
        
        
co = num_coroutine()
next(co)

for i in range(20):
    co.send(i)
    
co.close()  # 코루틴 종료

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
코루틴 종료


- 코루틴 안에서 예외 발생시키기

```
코루틴객체.throw(예외이름, 에러메시지)
```

In [7]:
def sum_coroutine():
    try:
        total = 0
        while True:
            x = (yield)
            total += x
    except RuntimeError as e:  
        print(e)
        yield total  # 코루틴 바깥으로 값 전달
        
        
co = sum_coroutine()
next(co)

for i in range(20):
    co.send(i)
    
print(co.throw(RuntimeError, '예외로 코루틴 끝내기'))

예외로 코루틴 끝내기
190


## 4. 하위 코루틴의 반환값 가져오기

- yield from에 코루틴을 지정하면 해당 코루틴에서 return으로 반환한 값을 가져옵니다.
- 코루틴에서 yield from을 사용하면 코루틴 바깥에서 send로 하위 코루틴까지 값을 보낼 수 있습니다.

```
변수 = yield from 코루틴()
```

In [9]:
def accumulate():
    total = 0
    while True:
        x = (yield)
        if x is None:  # 코루틴 바깥에서 값을 받아옴
            return total  # 받아온 값이 None이면
        total += x  # 합계 total을 반환
        
def sum_coroutine():
    while True:
        total = yield from accumulate()  # accumulate의 반환값을 가져옴
        print(total)

co = sum_coroutine()
next(co)

for i in range(1, 11):    # 1부터 10까지 반복
    co.send(i)            # 코루틴 accumulate에 숫자를 보냄
co.send(None)             # 코루틴 accumulate에 None을 보내서 숫자 누적을 끝냄
 
for i in range(1, 101):   # 1부터 100까지 반복
    co.send(i)            # 코루틴 accumulate에 숫자를 보냄
co.send(None)             # 코루틴 accumulate에 None을 보내서 숫자 누적을 끝냄
# sum_coroutine은 계속 돌고 있음

55
5050


### ※  StopIteration 예외 발생시키기
  - 코루틴도 제너레이터이므로 return을 사용하면 StopIteration이 발생합니다. 
  - 그래서 코루틴에서 return 값은 raise StopIteration(값)처럼 사용할 수도 있습니다(파이썬 3.6 이하).
  - 파이썬 3.7부터는 제너레이터 안에서 raise로 StopIteration 예외를 직접 발생시키면 RuntimeError로 바뀌므로 이 방법은 사용할 수 없습니다. 파이썬 3.7부터는 그냥 return 값을 사용해주세요

In [14]:
# 파이썬 3.7 이상이라 런타임 에러 발생
def accumulate():
    total = 0
    while True:
        x = (yield)
        if x is None:  # 코루틴 바깥에서 값을 받아옴
            raise StopIteration(total)  # 받아온 값이 None이면
        total += x  # 합계 total을 반환
        
def sum_coroutine():
    while True:
        total = yield from accumulate()  # accumulate의 반환값을 가져옴
        print(total)

co = sum_coroutine()
next(co)

for i in range(1, 11):    # 1부터 10까지 반복
    co.send(i)            # 코루틴 accumulate에 숫자를 보냄
co.send(None)             # 코루틴 accumulate에 None을 보내서 숫자 누적을 끝냄
 
for i in range(1, 101):   # 1부터 100까지 반복
    co.send(i)            # 코루틴 accumulate에 숫자를 보냄
co.send(None)             # 코루틴 accumulate에 None을 보내서 숫자 누적을 끝냄
# sum_coroutine은 계속 돌고 있음

RuntimeError: generator raised StopIteration

### ※ 코루틴의 yield from으로 값을 발생시키기

- 이번 예제에서는 x = (yield)와 같이 코루틴 바깥에서 보낸 값만 받아왔습니다. 하지만 코루틴에서 yield에 값을 지정해서 바깥으로 전달했다면 yield from은 해당 값을 다시 바깥으로 전달합니다.

In [16]:
def number_coroutine():
    x = None
    while True:
        x = (yield x)  # 코루틴 바깥에서 값을 받아오면서 바깥으로 값을 전달
        if x == 3:
            return x
        
def print_coroutine():
    while True:
        x = yield from number_coroutine()  # 하위 코루틴의 yield에 지정된 값을 다시 바깥으로 전달
        print('print_coroutine:', x)

co = print_coroutine()
next(co)

x = co.send(1)    # number_coroutine으로 1을 보냄
print(x)          # 1: number_coroutine의 yield에서 바깥으로 전달한 값
x = co.send(2)    # number_coroutine으로 2를 보냄
print(x)          # 2: number_coroutine의 yield에서 바깥으로 전달한 값
co.send(3)        # 3을 보내서 반환값을 출력하도록 만듦

1
2
print_coroutine: 3


## 41.7 심사문제 - 사칙연산 코루틴 만들기

In [29]:
def calc():
    result = None
    while True:
        expression = (yield result)
        result = eval(expression)

expressions = input().split(', ')
 
c = calc()
next(c)
 
for e in expressions:
    print(c.send(e))
 
c.close()

3 * 4, 10 / 5, 20 + 39
12
2.0
59
